In [141]:
import pandas as pd

photos = pd.read_csv('photos.csv',encoding='latin-1')
depeches = pd.read_csv('depeches.csv')


/Users/ya/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
photos.dropna(subset=['event'], inplace= True)


In [69]:
photos.shape

(245475, 45)

In [45]:
depeches.event.isnull().sum()

0

In [46]:
depeches.shape

(19975, 3)

In [47]:

print(photos.shape)
photos.event.isnull().sum()

(245475, 45)


0

Mettre en liste la colonne event. des deux bases 

In [70]:
c = 0 
for k in photos.event:    
    if k in depeches.event: 
        c+=1
    

In [76]:
liste_ev_unique_depeches = []
for ev in depeches.event:
    for _ev in ev[1:-1].replace("'","").split(', '):
        if _ev not in liste_ev_unique_depeches:
            liste_ev_unique_depeches.append(_ev)
print(len(liste_ev_unique_depeches))

13335


In [77]:
liste_ev_unique_photos = []
for ev in photos.event:
    for _ev in ev[1:-1].replace("'","").split(', '):
        if _ev not in liste_ev_unique_photos:
            liste_ev_unique_photos.append(_ev)
print(len(liste_ev_unique_photos))

16059


In [81]:
photos.columns

Index(['Unnamed: 0', 'Uno', 'caption', 'city', 'contributor', 'country',
       'country_only', 'country_out', 'created', 'creator', 'dept',
       'entity_company', 'entity_departement', 'entity_faces',
       'entity_function', 'entity_keyword', 'entity_location', 'entity_media',
       'entity_organisation', 'entity_person', 'entity_region', 'entity_video',
       'event', 'expires', 'genre', 'headline', 'iptc', 'keyword', 'language',
       'mediatopic', 'newsItemID', 'product', 'provider', 'published_date',
       'region', 'revision', 'rules', 'signal', 'slug', 'source', 'status',
       'title', 'topic', 'urgency', 'url_extracted'],
      dtype='object')

In [101]:
c = 0 
for k in photos.caption: 
    if len(k.split())>10 : 
        c+=1
c

153262

In [112]:
photos.index


Int64Index([     0,     12,     13,     14,     20,     22,     27,     33,
                34,     37,
            ...
            360975, 360977, 360978, 360980, 360982, 360984, 360987, 360988,
            360989, 360991],
           dtype='int64', length=153292)

In [152]:
photos.caption[12]

'President of the European Commission Ursula von der Leyen gestures during a special European Council summit in Brussels on February 20, 2020, held to discuss the next long-term budget of the European Union (EU). '

In [96]:
photos.shape

(245475, 45)

In [150]:
def author_del_photos(text):
    text_s = text
    while(text_s[-7:] != ' / AFP ' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-7])
photos.caption = photos.caption.apply(author_del_photos)          
photos.drop_duplicates(subset='caption', keep='first', inplace=True)

print(photos.shape)

(150536, 45)


In [153]:
import spacy 
import neuralcoref

#Toute cette cellule est issue de l'autre notebook (création du csv) et permet donc d'avoir toutes les fonctions
#définies dans ce dernier à disposition.


#On importe le fichier contenant les scores de chaque fonction grammaticale
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

#On importe un modèle md pour avoir des mots vectorisés
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def sort_ent(doc):
    """Retourne la liste des entités en les filtrant selon leur type et en les triant de manière à avoir au début de
    la liste les entités ayant des coréférences."""
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    """Retourne le score de chaque entité pour la méthode sans neuralcoref."""
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    """Retourne le score de chaque entité pour la méthode de base opérée sur le document resolved.
    Le document resolved est le document de base dans lequel toutes les références à un groupe de mot sont remplacées
    par celui-ci.
    Par exemple : My dad is home. He watches TV devient My dad is home. My dad watches TV.
    En raisonnant avec le nlp sur le document resolved, le nlp va détecter beaucoup plus de fois la même entité.
    Le principal inconvénient est que toutes les références sont remplacées, y compris celles qui ne sont pas associées 
    à des entités mais plutôt à des très longs bouts de phrase qui sont repris par un pronom comme "it"  """
    
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent_good_type(ent):
            if ent.text not in res.keys():
                res[ent.text]=dict_val[dep_ent(ent,doc)]
            else:
                res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False



def scores_doc_coref2(doc):
    """Cette méthode utilise neuralcoref mais au lieu d'agir sur le doc resolved, on va chercher a l'interieur des clusters
    associés au document. Pour rappel, un cluster contient un mot et ses références. Ce mot n'est pas forcément une entité.
    Un cluster est de la forme Trump : [Trump, he].
    Pour chaque entité, on va d'abord regarder si elle est coréférencée (donc mentionnée explicitement dans un cluster)
    ou pas.
    Si elle est coréférencée, on va chercher le cluster qui lui est associé.
    Une fois dans ce cluster, on va regarder pour chaque mention (réf à l'entité) si elle contient ou pas des entités.
    Si c'est le cas, on va alors indiquer que cette entité ne doit pas être retraitée par la suite car elle est traitée en
    tant qu'autre mention de l'entité que l'on traite actuellement.
    Par exemple, pour l'entité référencée Trump, on trouve le cluster suivant : Trump : [Trump, Donald Trump, he].
    Alors l'entité Donald Trump va être marquée comme traitée car elle est en fait équivalente à Trump.
    "he" n'étant pas une entité on a pas ce pb pour cette mention la.
    Ensuite on calcule le score de chaque mention (Trump, Donald Trump et he) en se basant sur leur fonction grammaticale.
    Puis on ajoute ces scores à celui de l'entité de base (Trump).
    
    Si l'entité de base n'est pas référencée, c'est un peu le même principe, la seule complexité en plus étant le fait que
    l'entité n'est pas forcément mentionnée telle quelle dans les mentions d'un cluster. 
    Par exemple, si notre entité de base est Sienna Miller, il est possible qu'on ait un cluster de la forme
    The british actress Sienna Miller : [The british actress Sienna Miller, she].
    Dans ce cas il faut rechercher si chaque mention contient l'entité de base avant de procéder comme pour les 
    entités référencées.
    
    """
    clusters=doc._.coref_clusters  #liste des clusters du doc. 
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents:
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(span,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(span,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    return res


def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)





/Users/ya/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/Users/ya/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)


In [155]:
from tqdm import tqdm_notebook


NLP = [nlp(dep) for dep in tqdm_notebook(photos.caption)]

/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [156]:
#scores_photos2 = [scores_doc_coref2(N) for N in tqdm_notebook(NLP)]
scores_photos1 = [scores_doc_coref1(N) for N in tqdm_notebook(NLP)]


/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [160]:
scores_photos1[10]

{'Austria': 0.38297872299999997,
 'Sebastian Kurz': 0.46808510600000003,
 'Luxembourg': 0.38297872299999997,
 'Xavier Bettel': 1.0,
 'European Council': 0.8297872340000001,
 'Brussels': 0.8297872340000001,
 'the European Union': 0.8297872340000001,
 'EU': 0.46808510600000003}

In [163]:

#global_ent_list = [[ent.text for ent in descr.ents if ent_good_type(ent)] for descr in tqdm_notebook(NLP)]

def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)

global_freq_list = [freq_dict(L) for L in tqdm_notebook(global_ent_list)]

/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [165]:
score_list_str1=[str(dic) for dic in tqdm_notebook(scores_photos1)]
#score_list_str2=[str(dic) for dic in tqdm(score_photos2)]

freq_list_str=[str(dic) for  dic in global_freq_list]
len(freq_list_str)

/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


150536

In [166]:
df_scores1=pd.DataFrame({'caption':photos.caption,'scores':score_list_str1,'freq':freq_list_str})
df_scores1.to_csv("scores_freq_photos1.csv",index=False)
#df_scores2=pd.DataFrame({'description':photos.caption,'scores':score_list_str2,'freq':freq_list_str})
#df_scores2.to_csv("scores_freq_photos2.csv",index=False)

In [167]:
df_scores1.head()

,caption,scores,freq
0,Portugal's Prime Minister Antonio Costa (2ndL)...,"{'Portugal': 0.38297872299999997, 'Antonio Cos...","{'the European Council': 0.125, 'European Coun..."
12,President of the European Commission Ursula vo...,{'the European Commission': 0.8297872340000001...,{'the European Commission': 0.1666666666666666...
13,Denmark's Prime Minister Mette Frederiksen (L)...,"{'Denmark': 0.38297872299999997, 'Mette Freder...","{'Denmark': 0.125, 'the European Council': 0.1..."
14,Police officers put up a vision barries in fro...,"{'Hanau': 1.6595744680000002, 'Germany': 1.297...","{'Hanau': 0.3333333333333333, 'Germany': 0.333..."
20,Denmark's Prime Minister Mette Frederiksen att...,"{'Denmark': 0.38297872299999997, 'Mette Freder...","{'Denmark': 0.16666666666666666, 'European Cou..."


In [1]:
import pandas as pd
score_photos = pd.read_csv("scores_freq_photos1.csv")
score_depeches = pd.read_csv("scores_freq_depeches.csv")


In [5]:
score_depeches

,scores,freq
0,{},"{'ISNA': 0.5, 'Iranian': 0.5}"
1,{},"{'Australia': 0.05405405405405406, 'Verreynne'..."
2,{},"{'Mahathir Mohamad': 0.14285714285714285, 'Anw..."
3,{},"{'Tottenham': 0.07142857142857142, 'Chelsea': ..."
4,{},{'the World Health Organization': 0.3333333333...
5,{},"{'Greek': 0.1111111111111111, 'Thessaloniki': ..."
6,{},"{'US': 0.2222222222222222, 'Phil Hogan': 0.111..."
7,{},"{'PAIGC': 0.2, 'Umaro Sissoco Embalo': 0.1, 'T..."
8,{},"{'SAS': 0.3333333333333333, 'Gustafson': 0.333..."
9,{},"{'Extinction Rebellion': 0.3333333333333333, '..."
